In [154]:
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from glob import glob
import re

In [155]:
path = "/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/"
df = pd.read_csv('../Annotations/processedAnnotations_no_corrupted_videos.csv')
count = df.groupby(['CM']).count()['palavra']
threshold = 20
frames = 10
cnn_dataset_path = "/home/fabiana/Desktop/projeto-final-src/Classifier/datasets/CNN"

In [156]:
def getFrameNumber(path):
    return int(re.search(r".*\/frame(\d+)\.jpg", path).group(1))

def getFiles(path):
    files = list(glob(path + "/*.jpg"))
    files.sort(key=getFrameNumber)
    return files

In [157]:
def replaceClass(cg):
    quantity = count[cg]
    if (quantity <= threshold):
        return 'others'
    else: 
        return cg
    
df['classe'] = df['CM'].map(replaceClass)
df['classe'] = df['classe'].astype('category')

In [158]:
df = df[['palavra', 'classe']]

In [159]:
df['first_letter'] = df['palavra'].map(lambda x: x[0])

In [160]:
df['folder_path'] = path + df['first_letter'] + '/' + df['palavra'] + f'/DEResult_{frames}/'

In [161]:
df['files_list'] = df['folder_path'].map(getFiles)

In [162]:
df.head()

,palavra,classe,first_letter,folder_path,files_list
0,A,cg01,A,/home/fabiana/Desktop/projeto-final-src/Crawle...,[/home/fabiana/Desktop/projeto-final-src/Crawl...
1,ABACATE,cg53a,A,/home/fabiana/Desktop/projeto-final-src/Crawle...,[/home/fabiana/Desktop/projeto-final-src/Crawl...
2,ABACAXI,cg47,A,/home/fabiana/Desktop/projeto-final-src/Crawle...,[/home/fabiana/Desktop/projeto-final-src/Crawl...
3,ABAFAR,cg07,A,/home/fabiana/Desktop/projeto-final-src/Crawle...,[/home/fabiana/Desktop/projeto-final-src/Crawl...
4,ABAIXO,cg62,A,/home/fabiana/Desktop/projeto-final-src/Crawle...,[/home/fabiana/Desktop/projeto-final-src/Crawl...


In [163]:
def get_images(images_list, img_size):
    frames_array = np.zeros((len(images_list), img_size[0], img_size[1]))
    for i, image_path in enumerate(images_list):
        img = image.load_img(image_path, color_mode='grayscale', target_size=img_size)
        img = np.squeeze(image.img_to_array(img))
        img /= 127.5
        img -= 1.
        frames_array[i,:,:] = img
    return frames_array

In [164]:
y = to_categorical(df['classe'].cat.codes.values)

In [172]:
X_train, X_test, y_train, y_test = train_test_split(df['files_list'], y, stratify=y,
                                                    test_size=0.2, random_state=0)

In [173]:
y_train = np.repeat(y_train, X_train.map(len), axis=0)

In [174]:
y_test = np.repeat(y_test, X_test.map(len), axis=0)

In [175]:
X_test_sum = np.array(X_test.sum())

In [176]:
X_train_sum = np.array(X_train.sum())

In [177]:
np.save(cnn_dataset_path + '/X_train.npy', X_train_sum)
np.save(cnn_dataset_path + '/X_test.npy', X_test_sum)
np.save(cnn_dataset_path + '/y_train.npy', y_train)
np.save(cnn_dataset_path + '/y_test.npy', y_test)

In [12]:
from tqdm import tqdm
X = []
for images_list in tqdm(df['files_list'].values):
    frames_array = get_images(images_list, img_size=(80, 100))
    X.append(np.expand_dims(frames_array, axis=0))
    
y = df['classe'].cat.codes

 47%|████▋     | 2694/5714 [08:48<12:08,  4.14it/s]

KeyboardInterrupt: 

In [ ]:
X.shape

In [92]:
frames_array.shape

(10, 176, 240)

In [82]:
img = image.load_img(image_path, color_mode='grayscale')

In [89]:
np.squeeze(image.img_to_array(img)).shape

(176, 240)

In [86]:
image.img_to_array(img).

(176, 240, 1)

In [63]:
def getImages(files_list):
    for image_path in files_list:
        img = image.load_img(image_path, target_size=(299, 299), color_mode='grayscale')
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

In [53]:
getFiles(df['folder_path'].iloc[0])

['/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame1.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame10.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame13.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame21.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame23.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame38.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame51.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame54.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame61.jpg',
 '/home/fabiana/Desktop/projeto-final-src/Crawler/Videos/A/A/DEResult_10/frame70.jpg']

In [12]:
labels = df['classe'].cat.codes

In [13]:
df.head()

,palavra,video_url,filename,classe_gramatical,assunto,configuracao_mao,CM,classe
0,A,http://www.acessibilidadebrasil.org.br/libras_...,A.mp4,SUBSTANTIVO,NENHUM,http://www.acessibilidadebrasil.org.br/libras_...,cg01,cg01
1,ABACATE,http://www.acessibilidadebrasil.org.br/libras_...,ABACATE.mp4,SUBSTANTIVO,FRUTA,http://www.acessibilidadebrasil.org.br/libras_...,cg53a,cg53a
2,ABACAXI,http://www.acessibilidadebrasil.org.br/libras_...,ABACAXI.mp4,SUBSTANTIVO,FRUTA,http://www.acessibilidadebrasil.org.br/libras_...,cg47,cg47
3,ABAFAR,http://www.acessibilidadebrasil.org.br/libras_...,ABAFAR.mp4,VERBO,NENHUM,http://www.acessibilidadebrasil.org.br/libras_...,cg07,cg07
4,ABAIXO,http://www.acessibilidadebrasil.org.br/libras_...,ABAIXO.mp4,ADV.,NENHUM,http://www.acessibilidadebrasil.org.br/libras_...,cg62,cg62
